In [1]:
#First let's load all the data so we can have a look at it. 

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pandas import json_normalize
import requests

# Starting with the historical weather data

In [3]:
weather = pd.read_csv('weather.csv')
weather.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1199145600,2008-01-01 00:00:00 +0000 UTC,0,Custom location,51.497256,-0.098179,9.07,5000.0,7.19,8.03,7.99,9.4,1027,NaN,NaN,88,2.1,270,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
1,1199149200,2008-01-01 01:00:00 +0000 UTC,0,Custom location,51.497256,-0.098179,8.92,5000.0,7.37,8.92,7.99,9.2,1026,NaN,NaN,90,1.0,260,NaN,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
2,1199152800,2008-01-01 02:00:00 +0000 UTC,0,Custom location,51.497256,-0.098179,8.69,4900.0,7.14,7.58,7.85,8.9,1026,NaN,NaN,90,2.1,210,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
3,1199156400,2008-01-01 03:00:00 +0000 UTC,0,Custom location,51.497256,-0.098179,8.69,4100.0,7.14,8.15,7.25,8.9,1025,NaN,NaN,90,1.5,210,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
4,1199160000,2008-01-01 04:00:00 +0000 UTC,0,Custom location,51.497256,-0.098179,9.00,3900.0,7.45,8.51,7.45,9.3,1025,NaN,NaN,90,1.5,220,NaN,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n


In [4]:
weather.shape

(35267, 28)

In [5]:
list(weather.columns)

['dt',
 'dt_iso',
 'timezone',
 'city_name',
 'lat',
 'lon',
 'temp',
 'visibility',
 'dew_point',
 'feels_like',
 'temp_min',
 'temp_max',
 'pressure',
 'sea_level',
 'grnd_level',
 'humidity',
 'wind_speed',
 'wind_deg',
 'wind_gust',
 'rain_1h',
 'rain_3h',
 'snow_1h',
 'snow_3h',
 'clouds_all',
 'weather_id',
 'weather_main',
 'weather_description',
 'weather_icon']

In [6]:
#cleaning up the file to remove columns with all/many NaNs, columns with no variance, etc.)

In [49]:
weather2 = weather.drop(['dt','timezone', 'city_name', 'lat', 'lon', 'sea_level', 'grnd_level', 'wind_gust', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h','visibility','dew_point'],axis=1)
weather2.head()

,dt_iso,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2008-01-01 00:00:00 +0000 UTC,9.07,8.03,7.99,9.4,1027,88,2.1,270,75,803,Clouds,broken clouds,04n
1,2008-01-01 01:00:00 +0000 UTC,8.92,8.92,7.99,9.2,1026,90,1.0,260,90,804,Clouds,overcast clouds,04n
2,2008-01-01 02:00:00 +0000 UTC,8.69,7.58,7.85,8.9,1026,90,2.1,210,75,803,Clouds,broken clouds,04n
3,2008-01-01 03:00:00 +0000 UTC,8.69,8.15,7.25,8.9,1025,90,1.5,210,75,803,Clouds,broken clouds,04n
4,2008-01-01 04:00:00 +0000 UTC,9.00,8.51,7.45,9.3,1025,90,1.5,220,90,804,Clouds,overcast clouds,04n


In [50]:
weather2.dtypes

dt_iso                  object
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
humidity                 int64
wind_speed             float64
wind_deg                 int64
clouds_all               int64
weather_id               int64
weather_main            object
weather_description     object
weather_icon            object
dtype: object

In [51]:
weather2.weather_main.value_counts(normalize=True)

Clouds          0.469050
Clear           0.350810
Rain            0.144129
Mist            0.017268
Drizzle         0.006919
Snow            0.004849
Haze            0.003743
Fog             0.002637
Thunderstorm    0.000595
Name: weather_main, dtype: float64

In [52]:
weather2.weather_icon.value_counts(normalize=True)

01n    0.211387
04n    0.143165
01d    0.139422
04d    0.119602
10d    0.075453
02d    0.074092
03d    0.068449
10n    0.053421
02n    0.035189
03n    0.028554
09d    0.017126
50d    0.016134
50n    0.007514
09n    0.005047
13n    0.002467
13d    0.002382
11d    0.000539
11n    0.000057
Name: weather_icon, dtype: float64

In [11]:
#Both columns with objects have a reasonably low number of categories so won't be a problem to categorize later on. 

In [53]:
round(weather2.isna().sum()/len(weather2),4)*100

dt_iso                 0.0
temp                   0.0
feels_like             0.0
temp_min               0.0
temp_max               0.0
pressure               0.0
humidity               0.0
wind_speed             0.0
wind_deg               0.0
clouds_all             0.0
weather_id             0.0
weather_main           0.0
weather_description    0.0
weather_icon           0.0
dtype: float64

In [13]:
#we had some NaNs in the visibility column, but column was dropped as not present in the "current weather" df. 

In [20]:
#Now let's tackle the dt_iso - remove part of the string and convert the rest to an actual datetime. 

In [21]:
weather2['dt_iso'] = weather2['dt_iso'].str[:-10]

In [22]:
weather2['dt_iso'] = pd.to_datetime(weather2['dt_iso'], format='%Y-%m-%d %H:%M:%S')

In [23]:
weather2.dtypes

dt_iso                 datetime64[ns]
temp                          float64
visibility                    float64
dew_point                     float64
feels_like                    float64
temp_min                      float64
temp_max                      float64
pressure                        int64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
clouds_all                      int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

In [24]:
#weather2.to_csv('weather_clean.csv',index=False)

In [25]:
#weather_clean = pd.read_csv('weather_clean.csv')

In [26]:
#weather_clean.head()

In [27]:
#weather_clean.dtypes

In [28]:
#after going to csv and back the dt_iso is back to an object! 

# Now the sales data

In [29]:
sales = pd.read_csv('sales.csv')
sales.head()

,Unnamed: 0,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0
1,1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0
2,2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0


In [30]:
sales = sales.drop(['Unnamed: 0'],axis=1)

In [31]:
sales.shape

(981330, 7)

In [32]:
sales = sales.drop_duplicates()

In [33]:
sales.shape

(948321, 7)

In [34]:
sales.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
dtype: object

In [35]:
sales['InvoiceDate'] = pd.to_datetime(sales['InvoiceDate'], format='%Y-%m-%d %H:%M:%S')

In [36]:
sales.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0


In [37]:
sales.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
dtype: object

# Now the current weather data/API call

In [38]:
import requests

In [39]:
#delete API key before uploading to Github!!! 

In [40]:
response = requests.get("http://api.openweathermap.org/data/2.5/find?q=London,UK&units=metric&appid=d0ea3794a0dc22414dc46fce5459e53d")

In [41]:
response

<Response [200]>

In [42]:
print(response.json())

{'message': 'accurate', 'cod': '200', 'count': 1, 'list': [{'id': 2643743, 'name': 'London', 'coord': {'lat': 51.5085, 'lon': -0.1257}, 'main': {'temp': 6.27, 'feels_like': 4.62, 'temp_min': 4.5, 'temp_max': 8.18, 'pressure': 1026, 'humidity': 65}, 'dt': 1646651353, 'wind': {'speed': 2.24, 'deg': 93}, 'sys': {'country': 'GB'}, 'rain': None, 'snow': None, 'clouds': {'all': 100}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}]}]}


In [43]:
current = response.text
current_df = pd.read_json(current, orient='records')
current_df

,message,cod,count,list
0,accurate,200,1,"{'id': 2643743, 'name': 'London', 'coord': {'l..."


In [44]:
flat1 = json_normalize(current_df['list'])
flat1_df = pd.DataFrame(flat1)
flat1_df = flat1_df.drop(['id','weather'],axis=1)
flat1_df

,name,dt,rain,snow,coord.lat,coord.lon,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,sys.country,clouds.all
0,London,1646651353,None,None,51.5085,-0.1257,6.27,4.62,4.5,8.18,1026,65,2.24,93,GB,100


In [45]:
weather_col = json_normalize(flat1['weather'][0])
weather_df = pd.DataFrame(weather_col)
weather_df

,id,main,description,icon
0,804,Clouds,overcast clouds,04d


In [46]:
current_weather = pd.concat([flat1_df,weather_df],axis=1)
current_weather

,name,dt,rain,snow,coord.lat,coord.lon,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,sys.country,clouds.all,id,main,description,icon
0,London,1646651353,None,None,51.5085,-0.1257,6.27,4.62,4.5,8.18,1026,65,2.24,93,GB,100,804,Clouds,overcast clouds,04d


In [ ]:
#Now we need to "format" the current weather df so it matches the weather 2 df for modelling later. 

In [55]:
list(weather2.columns)

['dt_iso',
 'temp',
 'feels_like',
 'temp_min',
 'temp_max',
 'pressure',
 'humidity',
 'wind_speed',
 'wind_deg',
 'clouds_all',
 'weather_id',
 'weather_main',
 'weather_description',
 'weather_icon']

In [48]:
list(current_weather.columns)

['name',
 'dt',
 'rain',
 'snow',
 'coord.lat',
 'coord.lon',
 'main.temp',
 'main.feels_like',
 'main.temp_min',
 'main.temp_max',
 'main.pressure',
 'main.humidity',
 'wind.speed',
 'wind.deg',
 'sys.country',
 'clouds.all',
 'id',
 'main',
 'description',
 'icon']

In [ ]:
#dropping columns from current_weather that are not present in the historical weather data/not needed

In [56]:
current_weather = current_weather.drop(['name',
 'dt',
 'rain',
 'snow',
 'coord.lat',
 'coord.lon','sys.country'], axis=1)

In [ ]:
#Now have to rename the columns so they match. 

In [58]:
current_weather = current_weather.rename(columns={
    'main.temp':'temp',
 'main.feels_like':'feels_like',
 'main.temp_min':'temp_min',
 'main.temp_max':'temp_max',
 'main.pressure':'pressure',
 'main.humidity':'humidity',
 'wind.speed':'wind_speed',
 'wind.deg':'wind_deg',
 'clouds.all':'clouds_all',
 'id':'weather_id',
 'main':'weather_main',
 'description':'weather_description',
 'icon':'weather_icon'})

In [60]:
current_weather

,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,6.27,4.62,4.5,8.18,1026,65,2.24,93,100,804,Clouds,overcast clouds,04d


In [ ]:
#Now putting all the current_weather cleaning into one function: 

In [61]:
def clean_current_weather(response):
    current = response.text
    current_df = pd.read_json(current, orient='records')
    flat1 = json_normalize(current_df['list'])
    flat1_df = flat1.drop(['id','weather'],axis=1)
    weather_df = json_normalize(flat1['weather'][0])
    current_weather = pd.concat([flat1_df,weather_df],axis=1)
    current_weather = current_weather.drop(['name','dt','rain','snow','coord.lat','coord.lon','sys.country'], axis=1)
    current_weather = current_weather.rename(columns={
    'main.temp':'temp',
    'main.feels_like':'feels_like',
    'main.temp_min':'temp_min',
    'main.temp_max':'temp_max',
    'main.pressure':'pressure',
    'main.humidity':'humidity',
    'wind.speed':'wind_speed',
    'wind.deg':'wind_deg',
    'clouds.all':'clouds_all',
    'id':'weather_id',
    'main':'weather_main',
    'description':'weather_description',
    'icon':'weather_icon'})
    return (current_weather)
current_weather = clean_current_weather(response)
current_weather

,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,6.27,4.62,4.5,8.18,1026,65,2.24,93,100,804,Clouds,overcast clouds,04d


# Now we join the sales and the historical weather information

In [ ]:
#Of course the date/time stamps of the sale don't match exactly the historical weather data (hourly record), so we'll have to "merge_asof"